In [7]:
import numpy as np
import pandas as pd
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, precision_score, accuracy_score
from sklearn.model_selection import KFold

In [ ]:
#import matplotlib.pyplot as plt
#%matplotlib inline
#plt.style.use('ggplot')

In [5]:
from src.split_dataframe import split_df
from src.remove_correlated_stats import remove_corr_stats
from src.my_predicted_stats import my_pred_stats

# EDA

In [ ]:
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df.describe()

In [ ]:
df.info()

### Split df_train in half to help with visualizing all of the data

In [ ]:
df_train_1 = df_train.iloc[:,3:12]
df_train_2 = df_train.iloc[:,12:20]
df_train_3 = df_train.iloc[:,20:30]
df_train_4 = df_train.iloc[:,30:40]
df_train_5 = df_train.iloc[:,40:]

In [ ]:
fig,ax = plt.subplots(figsize=(10,10))

ax = sns.heatmap(df_train_1.corr(), annot=True, cmap='coolwarm')
ax.set_title('Correlation Map')
plt.savefig('images/heat_map_1.png')

In [ ]:
fig,ax = plt.subplots(figsize=(10,10))

ax = sns.heatmap(df_train_2.corr(), annot=True, cmap='coolwarm')
ax.set_title('Correlation Map')
plt.savefig('images/heat_map_2.png')

In [ ]:
fig,ax = plt.subplots(figsize=(10,10))

ax = sns.heatmap(df_train_3.corr(), annot=True, cmap='coolwarm')
ax.set_title('Correlation Map')
plt.savefig('images/heat_map_3.png')

In [ ]:
fig,ax = plt.subplots(figsize=(10,10))

ax = sns.heatmap(df_train_4.corr(), annot=True, cmap='coolwarm')
ax.set_title('Correlation Map')
plt.savefig('images/heat_map_4.png')

In [ ]:
fig,ax = plt.subplots(figsize=(10,10))

ax = sns.heatmap(df_train_5.corr(), annot=True, cmap='coolwarm')
ax.set_title('Correlation Map')
plt.savefig('images/heat_map_5.png')

## Based off of the EDA, decided to drop the following columns:
columns_to_remove = ['passAttempts', 'passYardPerAtt', 'passIntPct', 'pass40Plus', 'sacks_allowed_yard','rush1stdowns', 'rush40plus']

# Random Forest Classifier - My Stats

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, precision_score, accuracy_score
from sklearn.model_selection import KFold

In [11]:
df = pd.read_csv('data/merged_games.csv')
df = df.sort_values(by=['season','week','game_id']).reset_index(drop=True)
df = remove_corr_stats(df)
my_df = my_pred_stats(df)

In [12]:
my_df.head()

,game_id,season,week,home_team,away_team,home_win_game,home_team_score,away_team_score,home_qb_rating,away_qb_rating,...,home_opponent_score,away_opponent_score,home_fumbles,away_fumbles,home_wins_past_games,away_wins_past_games,home_passTD,away_passTD,home_passInt,away_passInt
0,30348,2014,7,NE,NYJ,1,26.666667,16.000000,98.800000,69.516667,...,21.500000,26.333333,1.333333,2.166667,4,1,1.833333,1.000000,0.333333,1.000000
1,30349,2014,7,BUF,MIN,1,18.666667,17.333333,85.833333,71.050000,...,20.666667,23.833333,1.333333,0.833333,3,2,1.333333,0.500000,0.833333,1.333333
2,30352,2014,7,GB,CAR,1,27.166667,22.500000,104.350000,97.350000,...,21.333333,26.166667,1.000000,1.500000,4,3,2.500000,1.666667,0.500000,0.333333
3,30355,2014,7,WAS,TEN,1,22.000000,17.333333,90.466667,79.883333,...,27.666667,24.833333,1.333333,1.166667,1,2,1.666667,1.166667,1.333333,1.000000
4,30357,2014,7,BAL,ATL,1,27.333333,26.333333,101.400000,93.433333,...,16.166667,28.333333,1.000000,1.166667,4,2,2.000000,2.000000,0.500000,1.333333


In [14]:
my_df = my_df.sort_values(by=['season','week','game_id']).reset_index(drop=True)
df_train, df_test = split_df(my_df, 0.3)
y_train = np.array(df_train.pop('home_win_game'))
X_train = np.array(df_train.iloc[:,5:])
y_test = np.array(df_test.pop('home_win_game'))
X_test = np.array(df_test.iloc[:,5:])

### 10 fold split

In [15]:
kfold = KFold(n_splits=10)
accuracies = []
precisions = []
recalls = []
for train_index, test_index in kfold.split(X_train):
    model = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
    model.fit(X_train[train_index], y_train[train_index])
    y_predict = model.predict(X_train[test_index])
    y_true = y_train[test_index]
    accuracies.append(accuracy_score(y_true, y_predict))
    precisions.append(precision_score(y_true, y_predict))
    recalls.append(recall_score(y_true, y_predict))
print("Accuracy:", np.average(accuracies))
print("Precision:", np.average(precisions))
print("Recall:", np.average(recalls))

Accuracy: 0.6414553700267985
Precision: 0.6557873270633804
Recall: 0.7812306100738549


### Full model

In [18]:
model = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
model.fit(X_train, y_train)
y_predict = model.predict(X_test)
y_true = y_test
print("Accuracy:", accuracy_score(y_true, y_predict))
print("Precision:", precision_score(y_true, y_predict))
print("Recall:", recall_score(y_true, y_predict))

Accuracy: 0.6108490566037735
Precision: 0.6240601503759399
Recall: 0.7186147186147186


In [23]:
summarized_df = pd.DataFrame()

In [24]:
summarized_df['game_id'] = df_test['game_id']
summarized_df['season'] = df_test['season']
summarized_df['week'] = df_test['week']
summarized_df['home_team'] = df_test['home_team']
summarized_df['away_team'] = df_test['away_team']
summarized_df['home_team_predicted_win_perc'] = model.predict_proba(X_test)[:,1]
summarized_df['home_team_win'] = y_true

In [25]:
summarized_df

,game_id,season,week,home_team,away_team,home_team_predicted_win_perc,home_team_win
0,30348,2014,7,NE,NYJ,0.799257,1
1,30349,2014,7,BUF,MIN,0.690961,1
18,30374,2014,8,CLE,OAK,0.706415,1
19,30375,2014,8,PIT,IND,0.423982,1
34,30390,2014,9,NYG,IND,0.564816,0
...,...,...,...,...,...,...,...
1383,51695,2019,16,IND,CAR,0.559480,1
1396,51708,2019,17,DET,GB,0.343249,0
1397,51709,2019,17,HOU,TEN,0.311680,0
1398,51710,2019,17,JAX,IND,0.354107,1


In [26]:
feature_importances = pd.DataFrame(model.feature_importances_,
                                   index = df_train.columns[5:],
                                    columns=['importance']).sort_values('importance',ascending=False)

In [27]:
feature_importances

,importance
away_qb_rating,0.126410
home_team_score,0.122598
away_team_score,0.111623
home_qb_rating,0.095314
home_opponent_score,0.089680
home_wins_past_games,0.080852
away_passTD,0.052371
away_opponent_score,0.047757
away_wins_past_games,0.039858
away_fumbles,0.037356


# Random Forest Classifier - All Stats

In [30]:
df = pd.read_csv('data/merged_games.csv')
df = df.sort_values(by=['season','week','game_id']).reset_index(drop=True)
df = remove_corr_stats(df)

In [31]:
df = df.sort_values(by=['season','week','game_id']).reset_index(drop=True)
df_train, df_test = split_df(df, 0.3)
y_train = np.array(df_train.pop('home_win_game'))
X_train = np.array(df_train.iloc[:,5:])
y_test = np.array(df_test.pop('home_win_game'))
X_test = np.array(df_test.iloc[:,5:])

### 10 fold split

In [32]:
kfold = KFold(n_splits=10)
accuracies = []
precisions = []
recalls = []
for train_index, test_index in kfold.split(X_train):
    model = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
    model.fit(X_train[train_index], y_train[train_index])
    y_predict = model.predict(X_train[test_index])
    y_true = y_train[test_index]
    accuracies.append(accuracy_score(y_true, y_predict))
    precisions.append(precision_score(y_true, y_predict))
    recalls.append(recall_score(y_true, y_predict))
print("Accuracy:", np.average(accuracies))
print("Precision:", np.average(precisions))
print("Recall:", np.average(recalls))

Accuracy: 0.6053958177744586
Precision: 0.6173984703728458
Recall: 0.7465740488628432


### Full model

In [33]:
model = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
model.fit(X_train, y_train)
y_predict = model.predict(X_test)
y_true = y_test
print("Accuracy:", accuracy_score(y_true, y_predict))
print("Precision:", precision_score(y_true, y_predict))
print("Recall:", recall_score(y_true, y_predict))

Accuracy: 0.6666666666666666
Precision: 0.703862660944206
Recall: 0.7488584474885844


In [37]:
summarized_df = pd.DataFrame()

In [38]:
summarized_df['game_id'] = df_test['game_id']
summarized_df['season'] = df_test['season']
summarized_df['week'] = df_test['week']
summarized_df['home_team'] = df_test['home_team']
summarized_df['away_team'] = df_test['away_team']
summarized_df['home_team_predicted_win_perc'] = model.predict_proba(X_test)[:,1]
summarized_df['home_team_win'] = y_true

In [40]:
summarized_df

,game_id,season,week,home_team,away_team,home_team_predicted_win_perc,home_team_win
2,30352,2014,7,GB,CAR,0.547096,1
3,30355,2014,7,WAS,TEN,0.543776,1
12,30368,2014,8,NE,CHI,0.597850,1
13,30369,2014,8,NYJ,BUF,0.495182,0
16,30372,2014,8,JAX,MIA,0.423307,0
...,...,...,...,...,...,...,...
1377,51689,2019,16,NYJ,PIT,0.658462,1
1386,51698,2019,16,PHI,DAL,0.494773,1
1387,51699,2019,16,CHI,KC,0.433991,0
1392,51704,2019,17,NYG,PHI,0.498242,0


In [41]:
feature_importances = pd.DataFrame(model.feature_importances_,
                                   index = df_train.columns[5:],
                                    columns=['importance']).sort_values('importance',ascending=False)

In [43]:
feature_importances[0:10]

,importance
away_team_score,0.053481
home_opponent_score,0.040068
home_team_score,0.036784
away_qb_rating,0.035717
home_wins_past_games,0.033816
away_wins_past_games,0.026923
home_qb_rating,0.025213
away_passPct,0.023902
away_third_down_pct,0.023284
home_rushYards,0.022413


# OLD

In [ ]:
model = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
model.fit(X_train, y_train)
df_test['home_win_perc'] = model.predict_proba(X_test)[:,1]
df_test['home_win_game'] = y_test
summarized_df = df_test[['game_id','season', 'week','home_team','away_team','home_win_perc','home_win_game']]
summarized_df = summarized_df.sort_values(by='game_id').reset_index(drop=True)
summarized_df.reset_index(drop=True, inplace=True)
summarized_df['predicted_win'] = None
for i in range(0, len(summarized_df), 2):
    if summarized_df.loc[i, 'win_perc'] > summarized_df.loc[i+1, 'win_perc']:
        summarized_df.loc[i, 'predicted_win'] = 1
        summarized_df.loc[i+1, 'predicted_win'] = 0
    else:
        summarized_df.loc[i, 'predicted_win'] = 0
        summarized_df.loc[i+1, 'predicted_win'] = 1
y_true = np.array(summarized_df['win_game'])
y_predict = np.array(summarized_df['predicted_win'])
print("Accuracy:", accuracy_score(y_true, y_predict))
print("Precision:", precision_score(y_true, y_predict))
print("Recall:", recall_score(y_true, y_predict))